In [ ]:
import csv
import datetime
import math
import os
import os.path
import pdb
import re
import shutil
import subprocess
import sys
import time
from datetime import date
from multiprocessing import dummy as multiprocessing
from os.path import splitext
from shutil import copy

import cv2
import numpy as np
import pandas as pd
import pydicom as dicom
from pydicom.uid import UID, generate_uid

In [ ]:
def dicom_dataset_to_dict(dicom_header, fileToProcess):
    dicom_dict = {}
    dicom_dict["file"] = fileToProcess
    repr(dicom_header)
    for dicom_value in dicom_header.values():
        try:
            if dicom_value.tag == (0x0010, 0x0040):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0018, 0x1063):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0018, 0x0040):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x7FDF, 0x1074):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0028, 0x0008):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x18, 0x6011):
                for i in range(int(str(dicom_value)[-2])):
                    for dicom_value_2 in dicom_value[i]:
                        v = _convert_value(dicom_value_2.value)
                        if int(str(dicom_value)[-2]) > 1:
                            if str(dicom_value_2.tag) in dicom_dict.keys():
                                try:
                                    dicom_dict.setdefault(
                                        str(dicom_value_2.tag), []
                                    ).append(v)
                                except:
                                    (
                                        dicom_dict.setdefault(
                                            str(dicom_value_2.tag), []
                                        )
                                        + v
                                    )
                            else:
                                dicom_dict[str(dicom_value_2.tag)] = [v]
                        else:
                            dicom_dict[str(dicom_value_2.tag)] = v
            if dicom_value.tag == (0x0018, 0x1088):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0008, 0x0070):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0008, 0x0020):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0008, 0x0030):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0008, 0x1090):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0008, 0x1030):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0008, 0x1060):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0010, 0x0030):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0010, 0x1020):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0010, 0x1030):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0010, 0x21B0):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0010, 0x4000):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0028, 0x0004):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0010, 0x0020):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
            if dicom_value.tag == (0x0008, 0x0018):
                v = _convert_value(dicom_value.value)
                dicom_dict[str(dicom_value.tag)] = v
                continue
        except:
            continue
    return dicom_dict


def _sanitise_unicode(s):
    return s.replace("\u0000", "").strip()


def _convert_value(v):
    t = type(v)
    if t in (list, int, float):
        cv = v
    elif t == str:
        cv = _sanitise_unicode(v)
    elif t == bytes:
        s = v.decode("ascii", "replace")
        cv = _sanitise_unicode(s)
    elif t == dicom.valuerep.DSfloat:
        cv = float(v)
    elif t == dicom.valuerep.IS:
        cv = int(v)
    else:
        cv = repr(v)
    return cv

In [ ]:
def makeVideo(fileToProcess):
    try:
        fileName_2 = fileToProcess.split("/")[
            -2
        ]  # \\ if windows, / if on mac or sherlock
        fileName_1 = fileToProcess.split("/")[
            -1
        ]  # hex(abs(hash(fileToProcess.split('/')[-1]))).upper()
        fileName = fileName_2 + "_" + fileName_1
        dicom_dict = {}
        dataset = dicom.dcmread(fileToProcess, force=True)
        dicom_dict = dicom_dataset_to_dict(dataset, fileToProcess)
        return dicom_dict
    except:
        print("something failed, not sure what, have to debug", fileName)
    return 0

In [ ]:
from tqdm import tqdm

IMAGE_INDEX = "round6_mac_20200609_path.csv"
count = 0

final_list = []
with open(IMAGE_INDEX) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in tqdm(reader):
        count += 1
        VideoPath = os.path.join(row["image"])
        dicom_metadata = makeVideo(VideoPath)
        if isinstance(dicom_metadata, dict):
            final_list.append(dicom_metadata)
        if count % 50000 == 0:
            dicom_df_final = pd.DataFrame(final_list)
            dicom_df_final.to_csv("round6_mac_20210707_metadata.csv", index=False)
        else:
            pass
dicom_df_final = pd.DataFrame(final_list)
dicom_df_final.to_csv("round6_mac_20210707_metadata.csv", index=False)

In [ ]:
import pandas as pd

metadata = pd.read_csv("round6_mac_20210707_metadata.csv")
metadata = metadata[
    [
        "(0010, 0040)",
        "(0018, 1063)",
        "(0018, 0040)",
        "(7fdf, 1074)",
        "(0028, 0008)",
        "(0018, 602c)",
        "(0018, 602e)",
        "(0018, 1088)",
        "(0008, 0070)",
        "(0008, 0020)",
        "(0008, 0030)",
        "(0008, 1090)",
        "(0008, 1030)",
        "(0008, 1060)",
        "(0010, 0030)",
        "(0010, 1030)",
        "(0010, 1020)",
        "(0010, 21b0)",
        "(0010, 4000)",
        "(0028, 0004)",
        "(0010, 0020)",
        "(0008, 0018)",
        "file",
    ]
]

metadata.columns = [
    "sex",
    "FPS",
    "fps_2",
    "fps_3",
    "NumberOfFrames",
    "physical_delta_x",
    "physical_delta_y",
    "hr_bpm",
    "brand",
    "date",
    "time",
    "model",
    "study_type",
    "physician_reader",
    "birthdate",
    "patient_weight_kg",
    "patient_height_m",
    "patient_history",
    "patient_comments",
    "color_format",
    "mrn",
    "file_id",
    "FileName",
]

metadata["FPS"] = metadata["FPS"].fillna(metadata["fps_2"])
metadata["FPS"] = metadata["FPS"].fillna(metadata["fps_3"])
metadata["FPS"] = metadata["FPS"].fillna(30.0)
metadata = metadata.drop(columns=["fps_2", "fps_3"])
metadata["file_id"] = metadata["file_id"].str.replace("'", "")
metadata

In [ ]:
metadata.to_csv("round6_mac_20210707_metadata_clean.csv", index=False)